<a href="https://colab.research.google.com/github/ozgurural/SecurePoL-with-Watermarking/blob/main/pol_with_parameter_perturbation_based_watermark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%bash
DRIVE_ROOT=/content/drive/MyDrive
REPO_NAME=SecurePoL-with-Watermarking
REPO_DIR=$DRIVE_ROOT/$REPO_NAME
GIT_URL=https://github.com/ozgurural/SecurePoL-with-Watermarking.git

# assume /content/drive is already mounted via the Python cell
if [ ! -d "$DRIVE_ROOT" ]; then
  echo "ERROR: Drive not mounted. Please run the Python mount cell first."
  exit 1
fi

# Clone or pull
if [ ! -d "$REPO_DIR/.git" ]; then
  echo "Cloning repo into Drive…"
  git clone "$GIT_URL" "$REPO_DIR"
else
  echo "Updating repo…"
  cd "$REPO_DIR"
  git pull origin main
fi

# cd into your code
cd "$REPO_DIR/PoL"

Updating repo…
Updating 13ff14a..96e94b0
Fast-forward
 PoL/train.py                                       |   13 +-
 PoL/verify.py                                      |   75 +-
 PoL/watermark_utils.py                             |    2 +-
 pol_with_feature_based_watermark.ipynb             | 1118 +++++++++++++++-----
 ...th_parameter_perturbation_based_watermark.ipynb |  426 ++++++--
 5 files changed, 1248 insertions(+), 386 deletions(-)


From https://github.com/ozgurural/SecurePoL-with-Watermarking
 * branch            main       -> FETCH_HEAD
   13ff14a..96e94b0  main       -> origin/main


In [6]:
# Parameter-perturbation PoL + watermark (100 epochs, 20 params, Δ=1e-5)
!cd /content/drive/MyDrive/SecurePoL-with-Watermarking/PoL && \
python train.py \
  --dataset CIFAR10 \
  --model resnet20 \
  --epochs 10 \
  --save-freq 100 \
  --lr 0.1 \
  --watermark-method parameter_perturbation \
  --lambda-wm 0.01 \
  --k 1000 \
  --watermark-key secret_key \
  --num-parameters 20 \
  --perturbation-strength 1e-5 \
  --log-tb \
  --log-dir  /content/drive/MyDrive/SecurePoL-with-Watermarking/logs/param_pert \
  --model-dir /content/drive/MyDrive/SecurePoL-with-Watermarking/proof/CIFAR10_param_pert

2025-04-23 21:18:20,492 - INFO - train.py:510 - Attempting to allocate 1 GPU(s).
2025-04-23 21:18:20,497 - INFO - train.py:115 - Using device: cuda
2025-04-23 21:18:20,497 - INFO - train.py:116 - Data augmentation is disabled to ensure reproducibility for Proof-of-Learning.
2025-04-23 21:18:20,497 - INFO - train.py:121 - === Loading Dataset ===
2025-04-23 21:18:22,239 - INFO - train.py:124 - Dataset 'CIFAR10' loaded with 50000 samples.
2025-04-23 21:18:22,241 - INFO - train.py:131 - No sequence provided → training on subset_size=50000.
2025-04-23 21:18:22,347 - INFO - train.py:156 - Model: resnet20
2025-04-23 21:18:22,350 - INFO - train.py:181 - Optimizer: SGD, base LR=0.1
2025-04-23 21:18:24.092319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745443104.115503    2273 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to regi

In [ ]:
!python PoL/verify.py \
    --model-dir proof/CIFAR10_Batch100 \
    --dataset CIFAR10 \
    --model resnet20 \
    --epochs 2 \
    --save-freq 100 \
    --batch-size 128 \
    --lr 0.1 \
    --lambda-wm 0.01 \
    --k 1000 \
    --watermark-key 'secret_key' \
    --watermark-method 'parameter_perturbation' \
    --num-parameters 20 \
    --perturbation-strength 1e-10 \
    --dist 1 2 inf cos \
    --delta 10000 100 1 0.1 \
    --watermark-path model_with_parameter_perturbation_watermark.pth

2024-12-31 11:42:00,151 - INFO - Starting verification process...
2024-12-31 11:42:00,151 - INFO - Verifying model initialization (Kaiming, etc.)...
/content/SecurePoL-with-Watermarking/SecurePoL-with-Watermarking/PoL/verify.py:305: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control 